El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
# Importar bibliotecas
import pandas as pd # type: ignore
from IPython.display import display, HTML

In [2]:
# Cargar DataSet
df = pd.read_csv("datasets/car_data.csv")

In [3]:
# Visualizar datos 
display(HTML('<h1> Rusty Bargain DataFrame'))
display(HTML('<hr>'))
display(HTML('<h2> Informacion general'))
display(df.info())
display(HTML('<hr>'))
display(HTML('<h2> Muestra de tabla'))
display(df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
4955,28/03/2016 19:58,1250,NaN,1995,auto,0,3er,150000,0,petrol,bmw,NaN,28/03/2016 00:00,0,51143,07/04/2016 02:18
343788,19/03/2016 02:37,1000,sedan,1997,manual,116,primera,150000,1,NaN,nissan,no,18/03/2016 00:00,0,47803,28/03/2016 02:44
62509,28/03/2016 16:06,3490,wagon,2006,manual,109,NaN,150000,8,gasoline,ford,no,28/03/2016 00:00,0,56459,01/04/2016 03:18
38430,20/03/2016 15:06,8500,sedan,2006,auto,225,e_klasse,150000,0,gasoline,mercedes_benz,NaN,20/03/2016 00:00,0,56459,07/04/2016 00:44
280474,16/03/2016 17:51,4600,small,2005,manual,75,golf,150000,4,petrol,volkswagen,no,16/03/2016 00:00,0,26160,24/03/2016 10:45


In [5]:
# Revisar la cantidad de valores nulos en cada columna
missing_values = df.isnull().sum()

# Calcular el porcentaje de valores nulos
missing_percent = (missing_values / len(df)) * 100

# Resultados de valores faltantes
missing_data = pd.DataFrame({
    'Total Missing': missing_values,
    'Percent Missing (%)': missing_percent
})

# Filtrar columnas que tengan valores nulos
missing_data = missing_data[missing_data['Total Missing'] > 0].sort_values(by='Percent Missing (%)', ascending=False)

display(missing_data)


,Total Missing,Percent Missing (%)
NotRepaired,71154,20.079070
VehicleType,37490,10.579368
FuelType,32895,9.282697
Gearbox,19833,5.596709
Model,19705,5.560588


In [13]:
# Filtrar filas con valores nulos en ciertas columnas
filtered_df = df.dropna(subset=['VehicleType', 'FuelType', 'Gearbox', 'Model']).copy()

# Imputar la columna 'NotRepaired' con la categoría 'desconocido' usando .loc
filtered_df.loc[:, 'NotRepaired'] = filtered_df['NotRepaired'].fillna('desconocido')

# Verificar si quedan valores nulos
remaining_missing = filtered_df.isnull().sum()

# Mostrar las columnas que todavía tienen valores nulos
remaining_missing[remaining_missing > 0]

display(filtered_df.sample(5))


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
88465,09/03/2016 20:58,4950,wagon,2005,manual,140,passat,150000,11,gasoline,volkswagen,desconocido,09/03/2016 00:00,0,70569,05/04/2016 20:47
221732,16/03/2016 11:40,1050,convertible,1995,manual,90,other,150000,5,petrol,nissan,no,16/03/2016 00:00,0,95326,06/04/2016 04:16
252874,19/03/2016 17:47,9750,small,2010,auto,86,polo,20000,2,petrol,volkswagen,no,19/03/2016 00:00,0,23843,19/03/2016 17:47
135394,12/03/2016 17:45,16800,convertible,1981,manual,106,5_reihe,150000,6,petrol,peugeot,no,12/03/2016 00:00,0,79117,17/03/2016 20:21
34115,31/03/2016 19:53,4300,other,2007,manual,112,scenic,100000,4,petrol,renault,desconocido,31/03/2016 00:00,0,71679,31/03/2016 20:42


## Entrenamiento del modelo 

## Análisis del modelo

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos